# Übung 8: Feature Engineering und Parameter Tuning

## Aufgabe: Titanic reloaded

![](images/titanic.jpg)

Wie wir in Übung 4 gelernt haben gibt es noch einiges was wir tun können um die Perfromance im Titanic Dataset zu erhöhen. Unter anderem war FeatureEngineering und Parameter Tuning dabei. Dies lernen wir heute.

1. Schreiben Sie eine Funktion die die Passagiere der Titanic in die Altersklassen 0-16, 16-32, 32-48 und über 64 einteilt 
2. Erstellen Sie eine Funktion die die Anzahl der Familienmitglieder zählt und die Reisekosten pro Person
3. Erstellen Sie eine Funktion die die die Titel aus den Namen extrahiert. Fassen sie hierbei seltene Namen in eine Kategorie zusammen.
4. Benutzen Sie die Pipeline um die Funktionen aus Aufgabe 1-3 zu der aus Übung 4 bekannten Pipeline hinzuzufügen
5. Tunen Sie die Parameter eine DecisionTrees und vergleichen Sie die Ergebnisse mit der aus Übung 4

In [ ]:
import pandas as pd
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)

In [ ]:
titanic_train = pd.read_csv("data/titanic/train.csv")
titanic_train.head()

In [ ]:
def transform_age(df):
    X_temp = df.copy()
    # Implementerung hier

    return X_temp


age_transformer = FunctionTransformer(transform_age)

In [ ]:
def transform_family_fare(df):
    X_temp = df.copy()
    # Implementerung hier, erstellt Feature "Fcount", die die Familienmitglieder zählt und FarePerPerson, also das Entgelt geteilt durch die Familienmitglieder

    return X_temp


family_fare_transformer = FunctionTransformer(transform_family_fare)

In [ ]:
def transform_name(df):
    # Sir. / Mrs. etc..
    X_temp = df.copy()
    # Implementierung hier, erstellt das Feature Title, welches aus aus den Namen extrahiert werden kann.
    # Hinweis: der reguläre Ausdruck "\s*([A-Za-z]+)\." kann hilfreich sein

    return X_temp


name_transformer = FunctionTransformer(transform_name)
name_transformer.transform(titanic_train)

In [ ]:
# Hier können Sie sehen was das Feature Engineering gebracht hat
FeatureEngineering = Pipeline(
    steps=[
        # Details hier
    ]
)
FeatureEngineering.transform(titanic_train)

In [ ]:
ordinal_features = ["Sex"]
nominal_features = ["Embarked", "Title"]
numeric_features = ["Pclass", "Age", "Fare", "Fcount", "FarePerPerson"]

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)
nominal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoding", OneHotEncoder(handle_unknown="ignore")),
    ]
)
ordinal_transfomer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal_encoding", OrdinalEncoder()),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat_nominal", nominal_transformer, nominal_features),
        ("cat_ordinal", ordinal_transfomer, ordinal_features),
    ],
)

# Gesamtpipeline, es fehlen noch die Feature Engineering Transformer
preprocessor = Pipeline(
    steps=[
        ("column", column_transformer),
    ]
)

set_config(display="diagram")
preprocessor

In [ ]:
X = titanic_train.drop("Survived", axis=1)
y = titanic_train[["Survived"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train_prepared = preprocessor.fit_transform(X_train)
y_train_prepared = y_train.to_numpy().ravel()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {
    # Details einfügen
}

rf_clf = RandomForestClassifier()
rand_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=random_grid,
    n_iter=25,
    cv=3,
    random_state=42,
    n_jobs=-1,
)  # Fit the random search model
rand_search.fit(X_train_prepared, y_train_prepared)

rand_search.best_params_

## Load test data and make predictions

In [ ]:
X_test_prepared = preprocessor.transform(X_test)
clf = RandomForestClassifier(oob_score=True)
clf.fit(X_train_prepared, y_train_prepared)
print(f"Out of Bag Score ohne tuning: {clf.oob_score_}")
clf = RandomForestClassifier(**rand_search.best_params_, oob_score=True)
clf = clf.fit(X_train_prepared, y_train_prepared)
print(f"Out of Bag Score mit tuning: {clf.oob_score_}")

In [ ]:
predicted = clf.predict(X_test_prepared)

accuracy = accuracy_score(y_pred=predicted, y_true=y_test)
precision = precision_score(y_pred=predicted, y_true=y_test)
recall = recall_score(y_pred=predicted, y_true=y_test)
auc = roc_auc_score(y_true=y_test, y_score=predicted)
aps = average_precision_score(y_true=y_test, y_score=predicted)
f1 = f1_score(y_true=y_test, y_pred=predicted)

print(f"accuracy: {accuracy}")
print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC: {auc}")
print("\n")

## Evaluation

Vergleichen Sie die Perfromance mit Übung 4.

```
RandomForest
accuracy: 0.7737219730941704
precision: 0.7261904761904762
recall: 0.7261904761904762
F1 Score: 0.7261904761904762
AUC: 0.7803614251455979
```